In [1]:
import sys
#!{sys.executable} -m pip install pafy
#!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch
!apt-get install -y build-essential cmake
!apt-get install -y libopenblas-dev liblapack-dev
!{sys.executable} -m pip install dlib

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 101 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
Suggested packages:
  liblapack-doc
The following NEW packages will be installed:
  liblapack-dev liblapack3
0 upgraded, 2 newly installed, 0 to remove and 101 not upgraded.
Need to get 2764 kB of archives.
After this operation, 14.0 MB of additional disk space will be used.
Get:1 http://ports.ubuntu.com/ubuntu-ports bionic/main arm64 liblapack3 arm64 3.7.1-4ubuntu1 [1371 kB]
Get:2 http://ports.ubuntu.com/ubuntu-ports bionic/main arm64 liblapack-dev arm64 3.7.1-4ubuntu1 [1393 kB]
Fetched 2764 kB in 1s (3890 kB/s)   
debconf: delaying package configurat

In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
import traitlets
import PIL
from IPython.display import display, Image
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
from my_heatmap_generator import HMap
import cv2
import dlib

detector = dlib.cnn_face_detection_model_v1("./model/mmod_human_face_detector.dat")

device = torch.device('cuda')
mtcnn = MTCNN(keep_all=True, device=device)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()

print("model loaded")
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
load_dotenv()

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_duration = getenv('DEFAULT_DURATION')
radius = int(getenv('DEFAULT_RADIUS'))
duration = int(getenv('DEFAULT_DURATION'))

def on_value_change(change):
    print("Value change")
    radius = radius_slider.value
    duration = duration_slider.value

duration_slider = widgets.IntSlider(
    value=duration,
    min = 2,
    max = 30,
    step=1,
    description='Airborne Duration:',
    continuous_update=True,
    orientation='horizontal'
)

radius_slider = widgets.IntSlider(
    value=duration,
    min = 10,
    max = 200,
    step=1,
    description='Radius:',
    continuous_update=True,
    orientation='horizontal'
)

model loaded


In [2]:
HMAP = []
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0)
camera.running = True
camera.value
print("camera created")

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    global hmap, radius, duration
    frame = change['new']
    frame =cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #boxes, _ = mtcnn.detect(frame)
    results = detector(frame, 1)
    frame = PIL.Image.fromarray(frame)
    boxes = [r.rect for r in results]
    if boxes is not None:
        for box in boxes:
            rectangle = (box.left()-20, box.bottom()-20, box.right()+20, box.top()+20)
            face_only = frame.crop(rectangle)
            face_only = face_only.resize((224, 224))
            processed = preprocess(frame)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            radius = radius_slider.value
            duration = duration_slider.value
            hmap = HMap(box, category_index)
            if output[category_index] < 0.6:
                category_index = 0
            HMAP.append(hmap)
            frame = hmap.draw_map(frame, radius, duration)
    for hmap in HMAP:
        frame = hmap.draw_map(frame, radius, duration)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

camera.observe(execute, names='value')
display_handle = display(image_widget, widgets.HBox([duration_slider, radius_slider]))

camera created


Image(value=b'', format='jpeg', height='224', width='224')